In [1]:
import sys
sys.path.insert(1, '../Modules/')
from chunck import Chunck
from segmentator import Segmentator

In [2]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/', 
                 '../Models/shape_predictor_68_face_landmarks.dat', 1)

In [3]:
seg.segment_to_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4',
                       'TestDataSet/')

chunk:   3%|▎         | 153/5319 [00:00<00:03, 1441.37it/s, now=None]

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


NoBackendError: 

In [1]:
import os
import sys
import subprocess
import json
import math

import cv2
import vosk
import librosa
import pytube as pt
import numpy as np
import pandas as pd
import moviepy.editor as mp

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

sys.path.insert(1, '../Modules/')
from chunck import Chunck

In [8]:
class Segmentator:
	"""Класс для сегментации видоефайлов"""
	def __init__(self, audio_model_path, landmarks_model_path, conf=1):
		self.landmarks_model_path = landmarks_model_path
		self.audio_model_path = audio_model_path
		self.conf = conf # Вероятность, начиная с которой надо оставлять распознанные слов
		self.word_dict = None
		

	def segment_to_chuncks(self, path, out_path):
		"""
		Метод для сегментации видеофайла по словам в датасет

		path: путь для видео
		out_path: путь для датасета
		"""
		if self.word_dict is None:
			self.word_dict = self.segment_words(path)

		for key in self.word_dict:
			for index, word in enumerate(self.word_dict[key]):
				max_index = 0

				if word[2] >= self.conf:
					l_path = f'{out_path}/{word[0]}'
					w_path = f'{out_path}/{word[0]}/{word}'

					if not os.path.exists(l_path):
						os.mkdir(l_path)
					if not os.path.exists(w_path):
						os.mkdir(w_path)
					else:

						files = os.listdir()
						for file in files:
							max_index = max(max_index, file.partition('.'))

					temp_path = f'{w_path}/{max_index}t.mp4'
					ffmpeg_extract_subclip(path, word[0], word[1], temp_path)

					ch = Chunck(path, self.landmarks_model_path)
					ch.prepare()
					ch.to_file(f'{w_path}/{max_index}.mp4')

					if os.path.isfile(temp_path):
						os.remove(temp_path)
						

	def _extract_words(self, res):
		jres = json.loads(res)
		if not 'result' in jres:
			return []
		words = jres['result']
		return words

	def _transcribe_words(self, recognizer, bytes):
		result = []

		chunck_size = 4000
		for chunck_no in range(math.ceil(len(bytes)) / chunck_size):
			start = chunck_size * chunck_no
			end = min(len(bytes), (chunck_no + 1) * chunck_size)
			data = bytes[start : end]

			if recognizer.AcceptWaveform(data):
				words = self._extract_words(recognizer.Result())
				result += words

		result += self._extract_words(recognizer.FinalResult())

		return result

	def segment_words(self, input_path, out_path=None):
		"""
		Метод для сбора информации о таймингах слов в
		словарь или cvs файл

		input_path: путь к видеофайлу
		out_path: путь к csv файлу. Если None, то сохранять не надо

		returns: словарь слов с таймингами и вероятностями
		"""

		clip = mp.VideoFileClip(input_path)
		temp_audio_path = 'audio.mp3'
		clip.audio.write_audiofile(temp_audio_path)

		vosk.SetLogLevel(-1)

		sample_rate = 16000
		audio, sr = librosa.load(temp_audio_path, sr=sample_rate)

		int16 = np.int16(audio * 32768).tobytes()

		model = vosk.Model(self.audio_model_path)
		recognizer = vosk.KaldiRecognizer(model, sample_rate)

		res = transcribe_words(recognizer, int16)
		df = pd.DataFrame.from_records(res)
		df = df.sort_values('start')

		if out_path is not None:
			df.to_cvs(out_path, index=False)


		word_dict = {}
		for index, row in df.iterrows():
			word_dict[row['word']] = []

		for index, row in df.iterrows():
			word_dict[row['word']].append((row['start'], row['end'], row['conf']))

		if os.path.isfile(temp_audio_path):
			os.delete(temp_audio_path)

		return word_dict

In [9]:
s = Segmentator('../Models/vosk-model-small-ru-0.4/',
               '../Models/shape_predictor_68_face_landmarks.dat')

In [10]:
s.segment_to_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4', 
                     'TestDataSet/')

chunk:   3%|▎         | 153/5319 [00:00<00:03, 1471.72it/s, now=None]

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


NoBackendError: 

In [11]:
s.segment_words('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4')

chunk:   2%|▏         | 132/5319 [00:00<00:03, 1319.60it/s, now=None]

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


NoBackendError: 

In [33]:
import sys
import os
import subprocess
import json
import math

import vosk
import librosa
import numpy as np
import pandas as pd
import moviepy.editor as mp

In [39]:
class Segmentator:
    def __init__(self, audio_model_path, video_model_path,
                conf=1):
        self.audio_model_path = audio_model_path
        self.video_model_path = video_model_path
        self.conf = conf
        
    def _to_dict(self, df):
        result = {}
        for index, row in df.iterrows():
            if row['conf'] >= self.conf:
                result[row['word']] = []

        for index, row in df.iterrows():
            if row['conf'] >= self.conf:
                result[row['word']].append((row['start'], row['end']))
        return result
        
    def _extract_words(self, res):
        jres = json.loads(res)
        if not 'result' in jres:
            return []
        words = jres['result']
        return words
    
    def _transcribe_words(self, recognizer, bytes):
        result = []

        chunk_size = 4000
        for chunk_no in range(math.ceil(len(bytes) / chunk_size)):
            start = chunk_no * chunk_size
            end = min(len(bytes), (chunk_no + 1) * chunk_size)
            data = bytes[start : end]

            if recognizer.AcceptWaveform(data):
                words = self._extract_words(recognizer.Result())
                result += words
        result += self._extract_words(recognizer.FinalResult())

        return result
    
    def get_info(self, input_path, out_path=None):
        vosk.SetLogLevel(-1)
        
        temp_audio_path = 'temp.wav'
        clip = mp.VideoFileClip(input_path)
        clip.audio.write_audiofile(temp_audio_path)

        sample_rate = 16000
        audio, sr = librosa.load(temp_audio_path, sr=sample_rate)

        int16 = np.int16(audio * 32768).tobytes()

        model = vosk.Model(self.audio_model_path)
        recognizer = vosk.KaldiRecognizer(model, sample_rate)

        res = self._transcribe_words(recognizer, int16)
        df = pd.DataFrame.from_records(res)
        df = df.sort_values('start')
        
        if os.path.isfile(temp_audio_path):
            os.remove(temp_audio_path)
        
        if out_path is not None:
            df.to_csv(out_path, index=False)
            
        return self._to_dict(df)
    

In [40]:
s = Segmentator('../Models/vosk-model-small-ru-0.4/', '')

In [41]:
d = s.get_info('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4')

chunk:   5%|▌         | 281/5319 [00:00<00:01, 2809.90it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.


In [42]:
d

{'привет': [(0.905369, 1.2)],
 'ещё': [(1.2, 1.41), (123.45, 123.75), (207.27, 207.57)],
 'раз': [(1.41, 1.59), (225.78, 226.02)],
 'расскажу': [(1.71, 2.43), (226.05, 226.5)],
 'началось': [(3.96, 4.5011)],
 'две': [(4.62, 5.01)],
 'идеи': [(5.01, 5.49)],
 'поделиться': [(6.75, 7.23)],
 'первая': [(7.62, 8.4), (14.67, 15.149999)],
 'то': [(8.4, 8.64)],
 'что': [(8.64, 9.21),
  (25.32, 25.53),
  (62.52, 62.7),
  (100.98, 101.55),
  (113.97, 114.15),
  (218.97, 219.12)],
 'с': [(9.27, 9.45),
  (12.66, 12.72),
  (66.54, 66.63),
  (69.78, 69.87),
  (117.42, 117.84),
  (227.52, 227.61)],
 'этого': [(9.45, 9.72)],
 'года': [(9.72, 9.93)],
 'в': [(9.93, 9.99),
  (38.91, 38.940421),
  (74.7, 75.36),
  (110.46, 110.52),
  (129.87, 129.93),
  (150.0, 150.06),
  (166.53, 166.59)],
 'шаге': [(9.99, 10.32), (96.36, 96.9), (110.52, 111.06), (150.06, 150.36)],
 'есть': [(10.32, 10.62), (70.59, 70.8), (130.11, 130.29)],
 'четыре': [(10.62, 11.13)],
 'направления': [(11.13, 11.85), (90.27, 90.93)],
 '